# Bayesian network baselines

We build two Bayesian network baselines. 

![title](figures/models_BN.png)

## BN baseline

We initialize the structure of the BN using the true ground truth DAG. We learn the conditional probability distributions that specify the joint probability distribution from the training data. We evaluate diagnostic performance through Bayesian inference on the test data. 

In [3]:
%load_ext autoreload
%autoreload 2

In [1]:
import pickle
with open("data/train_4000_final.p", "rb") as file: 
    train_df = pickle.load(file)

In [2]:
import pickle
with open("data/test_1000_final.p", "rb") as file: 
    test_df = pickle.load(file)

In [7]:
from pgmpy.inference.ExactInference import VariableElimination
import pandas as pd

def get_diagnosis_pred_symptom(model, test_set, excl_sympt=False):
    """
    build prediction dataframe for diagnoses (pneu, inf) based on all observed symptoms (dysp, cough, nasal) and background (season)
    P(Pneu=yes|Symptoms,Season) and P(Inf=yes|Symptoms,Season)

    model: BN model with learned probability tables 
    test_set: dataframe with test cases
    excl_sympt: whether to exclude Symptoms from evidence, and instead calculate P(Pneu=yes|Season) and P(Inf=yes|Season)

    returns: test_set dataframe, extended with prediction for diagnoses (pred_pneu, pred_inf)
    """

    res_df = pd.DataFrame(columns=list(test_set.columns))
    inf = VariableElimination(model)

    for i in test_set.index: 
        x = test_set.loc[i]
        if not x.isna().any(): # skip over cases where symptoms are unknown
            evidence = {key:val for key, val in x.items() if key not in ["pneu", "inf"]}
            if not excl_sympt:
                res_pneu = inf.query(["pneu"], evidence=evidence, show_progress=False).get_value(**{"pneu":"yes"}) # P(Pneu = 1 | Season, Dysp, Cough, Nasal)
                res_inf = inf.query(["inf"], evidence=evidence, show_progress=False).get_value(**{"inf":"yes"}) # P(Inf = 1 | Season, Dysp, Cough, Nasal)
            else: 
                res_pneu = inf.query(["pneu"], evidence={"season": x["season"]}, show_progress=False).get_value(**{"pneu":"yes"})
                res_inf = inf.query(["inf"], evidence={"season": x["season"]}, show_progress=False).get_value(**{"inf":"yes"})
            batch_df = pd.DataFrame(x.to_dict(), index=[i])
            batch_df["pred_pneu"] = res_pneu
            batch_df["pred_inf"] = res_inf
            res_df = pd.concat([res_df, batch_df], ignore_index=False)

    return res_df

In [12]:
from utils.baselines import fit_BN_model
from utils.evaluation import CPD_params, performance_metrics

model_type = "BN"

test_set = test_df[["season", "pneu", "inf", "dysp", "cough", "nasal"]]
train_set = train_df[["season", "pneu", "inf", "dysp", "cough", "nasal"]]

results = {}

model = fit_BN_model(train_set)

_, _, results["KL"] = CPD_params(model, model_type)

pred_df = get_diagnosis_pred_symptom(model, test_set, excl_sympt=False)
roc_auc, pr_auc = performance_metrics(pred_df, "pneu", model_type, plot=False)
results["P(pneu|season,symptoms) test PR"] = pr_auc
roc_auc, pr_auc = performance_metrics(pred_df, "inf", model_type, plot=False)
results["P(inf|season,symptoms) train PR"] = pr_auc

pred_df = get_diagnosis_pred_symptom(model, test_set, excl_sympt=True)
roc_auc, pr_auc = performance_metrics(pred_df, "pneu", model_type, plot=False)
results["P(pneu|season) test PR"] = pr_auc
roc_auc, pr_auc = performance_metrics(pred_df, "inf", model_type, plot=False)
results["P(inf|season) train PR"] = pr_auc


In [13]:
results

{'KL': 0.0028532112739444327,
 'P(pneu|season,symptoms) test PR': 0.0914367231372981,
 'P(inf|season,symptoms) train PR': 0.8883862289669199,
 'P(pneu|season) test PR': 0.03022864060878589,
 'P(inf|season) train PR': 0.4441163639127461}

## BN++ Baseline

This Bayesian network is trained with a version of the dataset where pain and fever are exceptionally *not* masked out. We also evaluate on a version of the test set where the two are unmasked, meaning they are included in the evidence during inference. 

In [2]:
import pickle
with open("data/train_4000_final_fever_pain.p", "rb") as file: 
    train_df = pickle.load(file)

In [1]:
import pickle
with open("data/test_1000_final_fever_pain.p", "rb") as file: 
    test_df = pickle.load(file)

This model needs its own implementation of the CPD_params function, since the one in utils/evaluation assumes that the fever and pain symptoms are missing. 

In [3]:
from pgmpy.inference.ExactInference import VariableElimination
import itertools
import pandas as pd
import numpy as np

def calc_prob_CPT_BN_plus(df, combo): 
    val = {"season": combo[0], "pneu": combo[1], "inf": combo[2], "dysp": combo[3], "cough": combo[4], "nasal":combo[5], "pain": combo[6], "fever":combo[7]} 
    prob = df["season"].iloc[val["season"], 0] 
    prob *= df["pneu"].iloc[val["pneu"], val["season"]]
    prob *= df["inf"].iloc[val["inf"], val["season"]]
    prob *= df["dysp"].iloc[val["dysp"], val["pneu"]]
    prob *= df["nasal"].iloc[val["nasal"], val["inf"]]
    idx = 2*(val["pneu"]) + val["inf"]
    prob *= df["cough"].iloc[val["cough"], idx]
    prob *= df["fever"].iloc[val["fever"], idx]
    prob *= df["pain"].iloc[val["pain"], idx]
    return prob

def CPD_params_BN_plus(model): 

    # ground truth CPD parameters from BN
    df_GT = {}
    df_GT["season"] = pd.DataFrame({"": [0.4, 0.6]}, index=["season = cold", "season = warm"])
    df_GT["pneu"] = pd.DataFrame({"season = warm": [0.005, 0.995], 
                            "season = cold": [0.015, 0.985]}, index=["pneu = yes", "pneu = no"])
    df_GT["inf"] = pd.DataFrame({"season = warm": [0.05, 0.95], 
                           "season = cold": [0.5, 0.5]}, index=["inf = yes", "inf = no"])
    df_GT["dysp"] = pd.DataFrame({"pneu = yes": [0.3, 0.7],
                            "pneu = no": [0.15, 0.85]}, index=["dysp = yes", "dysp = no"])
    df_GT["nasal"] = pd.DataFrame({"inf = yes": [0.7, 0.3], 
                             "inf = no": [0.2, 0.8]}, index=["nasal = yes", "nasal = no"])
    df_GT["cough"] = pd.DataFrame({"pneu = yes, inf = yes": [0.9, 0.1], 
                             "pneu = yes, inf = no": [0.9, 0.1], 
                             "pneu = no, inf = yes": [0.8, 0.2], 
                             "pneu = no, inf = no": [0.05, 0.95]}, index=["cough = yes", "cough = no"])
    df_GT["fever"] = pd.DataFrame({"pneu = yes, inf = yes": [0.80, 0.15, 0.05], 
                             "pneu = yes, inf = no": [0.80, 0.10, 0.10], 
                             "pneu = no, inf = yes": [0.01, 0.14, 0.85], 
                             "pneu = no, inf = no": [0.05, 0.15, 0.80]}, index=["fever = high", "fever = low", "fever = none"])
    df_GT["pain"] = pd.DataFrame({"pneu = yes, inf = yes": [0.3, 0.7], 
                             "pneu = yes, inf = no": [0.3, 0.7], 
                             "pneu = no, inf = yes": [0.1, 0.9], 
                             "pneu = no, inf = no": [0.05, 0.95]}, index=["pain = yes", "pain = no"])

    # CPD parameters estimated from BN
    inf = VariableElimination(model)

    seas_prob = inf.query(["season"], evidence={}, show_progress=False).get_value(**{"season":"cold"})

    pneu_prob = [0, 0]
    pneu_prob[0] = inf.query(["pneu"], evidence={"season": "warm"}, show_progress=False).get_value(**{"pneu":"yes"})
    pneu_prob[1] = inf.query(["pneu"], evidence={"season": "cold"}, show_progress=False).get_value(**{"pneu":"yes"})

    inf_prob = [0, 0]
    inf_prob[0] = inf.query(["inf"], evidence={"season": "warm"}, show_progress=False).get_value(**{"inf":"yes"})
    inf_prob[1] = inf.query(["inf"], evidence={"season": "cold"}, show_progress=False).get_value(**{"inf":"yes"})

    dysp_prob = [0, 0]
    dysp_prob[0] = inf.query(["dysp"], evidence={"pneu": "no"}, show_progress=False).get_value(**{"dysp":"yes"})
    dysp_prob[1] = inf.query(["dysp"], evidence={"pneu": "yes"}, show_progress=False).get_value(**{"dysp":"yes"})

    nasal_prob = [0, 0]
    nasal_prob[0] = inf.query(["nasal"], evidence={"inf": "no"}, show_progress=False).get_value(**{"nasal":"yes"})
    nasal_prob[1] = inf.query(["nasal"], evidence={"inf": "yes"}, show_progress=False).get_value(**{"nasal":"yes"})

    cough_prob = [0, 0, 0, 0]
    cough_prob[0] = inf.query(["cough"], evidence={"pneu": "no", "inf": "no"}, show_progress=False).get_value(**{"cough":"yes"})
    cough_prob[1] = inf.query(["cough"], evidence={"pneu": "no", "inf": "yes"}, show_progress=False).get_value(**{"cough":"yes"})
    cough_prob[2] = inf.query(["cough"], evidence={"pneu": "yes", "inf": "no"}, show_progress=False).get_value(**{"cough":"yes"})
    cough_prob[3] = inf.query(["cough"], evidence={"pneu": "yes", "inf": "yes"}, show_progress=False).get_value(**{"cough":"yes"})

    fever_prob = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    fever_prob[0] = inf.query(["fever"], evidence={"pneu": "no", "inf": "no"}, show_progress=False).get_value(**{"fever":"none"})
    fever_prob[1] = inf.query(["fever"], evidence={"pneu": "no", "inf": "no"}, show_progress=False).get_value(**{"fever":"low"})
    fever_prob[2] = inf.query(["fever"], evidence={"pneu": "no", "inf": "no"}, show_progress=False).get_value(**{"fever":"high"})
    fever_prob[3] = inf.query(["fever"], evidence={"pneu": "no", "inf": "yes"}, show_progress=False).get_value(**{"fever":"none"})
    fever_prob[4] = inf.query(["fever"], evidence={"pneu": "no", "inf": "yes"}, show_progress=False).get_value(**{"fever":"low"})
    fever_prob[5] = inf.query(["fever"], evidence={"pneu": "no", "inf": "yes"}, show_progress=False).get_value(**{"fever":"high"})
    fever_prob[6] = inf.query(["fever"], evidence={"pneu": "yes", "inf": "no"}, show_progress=False).get_value(**{"fever":"none"})
    fever_prob[7] = inf.query(["fever"], evidence={"pneu": "yes", "inf": "no"}, show_progress=False).get_value(**{"fever":"low"})
    fever_prob[8] = inf.query(["fever"], evidence={"pneu": "yes", "inf": "no"}, show_progress=False).get_value(**{"fever":"high"})
    fever_prob[9] = inf.query(["fever"], evidence={"pneu": "yes", "inf": "yes"}, show_progress=False).get_value(**{"fever":"none"})
    fever_prob[10] = inf.query(["fever"], evidence={"pneu": "yes", "inf": "yes"}, show_progress=False).get_value(**{"fever":"low"})
    fever_prob[11] = inf.query(["fever"], evidence={"pneu": "yes", "inf": "yes"}, show_progress=False).get_value(**{"fever":"high"})

    pain_prob = [0, 0, 0, 0]
    pain_prob[0] = inf.query(["pain"], evidence={"pneu": "no", "inf": "no"}, show_progress=False).get_value(**{"pain":"yes"})
    pain_prob[1] = inf.query(["pain"], evidence={"pneu": "no", "inf": "yes"}, show_progress=False).get_value(**{"pain":"yes"})
    pain_prob[2] = inf.query(["pain"], evidence={"pneu": "yes", "inf": "no"}, show_progress=False).get_value(**{"pain":"yes"})
    pain_prob[3] = inf.query(["pain"], evidence={"pneu": "yes", "inf": "yes"}, show_progress=False).get_value(**{"pain":"yes"})
 
    df_est = {}
    df_est["season"] = pd.DataFrame({"": [seas_prob, 1-seas_prob]}, index=["season = cold", "season = warm"])
    df_est["pneu"] = pd.DataFrame({"season = warm": [pneu_prob[0], 1-pneu_prob[0]], 
                                   "season = cold": [pneu_prob[1], 1-pneu_prob[1]]}, index=["pneu = yes", "pneu = no"])
    df_est["inf"] = pd.DataFrame({"season = warm": [inf_prob[0], 1-inf_prob[0]], 
                                  "season = cold": [inf_prob[1], 1-inf_prob[1]]}, index=["inf = yes", "inf = no"])
    df_est["dysp"] = pd.DataFrame({"pneu = yes": [dysp_prob[1], 1-dysp_prob[1]],
                                   "pneu = no": [dysp_prob[0], 1-dysp_prob[0]]}, index=["dysp = yes", "dysp = no"])
    df_est["nasal"] = pd.DataFrame({"inf = yes": [nasal_prob[1], 1-nasal_prob[1]], 
                                    "inf = no": [nasal_prob[0], 1-nasal_prob[0]]}, index=["nasal = yes", "nasal = no"])
    df_est["cough"] = pd.DataFrame({"pneu = yes, inf = yes": [cough_prob[3], 1-cough_prob[3]], 
                                    "pneu = yes, inf = no": [cough_prob[2], 1-cough_prob[2]], 
                                    "pneu = no, inf = yes": [cough_prob[1], 1-cough_prob[1]], 
                                    "pneu = no, inf = no": [cough_prob[0], 1-cough_prob[0]]}, index=["cough = yes", "cough = no"])
    df_est["fever"] = pd.DataFrame({"pneu = yes, inf = yes": [fever_prob[11], fever_prob[10], fever_prob[9]], 
                                    "pneu = yes, inf = no": [fever_prob[8], fever_prob[7], fever_prob[6]], 
                                    "pneu = no, inf = yes": [fever_prob[5], fever_prob[4], fever_prob[3]], 
                                    "pneu = no, inf = no": [fever_prob[2], fever_prob[1], fever_prob[0]]}, index=["fever = high", "fever = low", "fever = none"])
    df_est["pain"] = pd.DataFrame({"pneu = yes, inf = yes": [pain_prob[3], 1-pain_prob[3]], 
                                    "pneu = yes, inf = no": [pain_prob[2], 1-pain_prob[2]], 
                                    "pneu = no, inf = yes": [pain_prob[1], 1-pain_prob[1]], 
                                    "pneu = no, inf = no": [pain_prob[0], 1-pain_prob[0]]}, index=["pain = yes", "pain = no"])

    # calculate KL divergence based on joint probability distribution for both CPD configurations
    KL =0
    val_combos = list(itertools.product([0, 1], repeat=7))
    all_combos = []
    for combo in val_combos: # add pain (three categories)
        for j in (0, 1, 2): 
            combo_l = list(combo)
            combo_l.append(j)
            all_combos.append(combo_l)
    for combo in all_combos: # sum over all possible x
        prob_GT = calc_prob_CPT_BN_plus(df_GT, combo) # P(x)
        prob_est = calc_prob_CPT_BN_plus(df_est, combo) # Q(x)
        if prob_est == 0: 
            prob_est += 1e-8 # avoid division by zero error
        KL += prob_GT*np.log(prob_GT/prob_est) # P(x)*log(P(x)/Q(x))

    return df_GT, df_est, KL

c:\Users\Paloma\anaconda3\envs\bn-text\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from utils.baselines import fit_BN_plus_model
from utils.evaluation import performance_metrics

model_type = "BN_plus"

test_set = test_df[["season", "pneu", "inf", "dysp", "cough", "fever", "pain", "nasal"]]
train_set = train_df[["season", "pneu", "inf", "dysp", "cough", "fever", "pain", "nasal"]]

results = {}

model = fit_BN_plus_model(train_set)

_, _, results["KL"] = CPD_params_BN_plus(model)

pred_df = get_diagnosis_pred_symptom(model, test_set, excl_sympt=False)
roc_auc, pr_auc = performance_metrics(pred_df, "pneu", model_type, plot=False)
results["P(pneu|season,symptoms) test PR"] = pr_auc
roc_auc, pr_auc = performance_metrics(pred_df, "inf", model_type, plot=False)
results["P(inf|season,symptoms) train PR"] = pr_auc

pred_df = get_diagnosis_pred_symptom(model, test_set, excl_sympt=True)
roc_auc, pr_auc = performance_metrics(pred_df, "pneu", model_type, plot=False)
results["P(pneu|season) test PR"] = pr_auc
roc_auc, pr_auc = performance_metrics(pred_df, "inf", model_type, plot=False)
results["P(inf|season) train PR"] = pr_auc


In [9]:
results

{'KL': 0.005035791416727543,
 'P(pneu|season,symptoms) test PR': 0.8326185019462329,
 'P(inf|season,symptoms) train PR': 0.9008519220159485,
 'P(pneu|season) test PR': 0.03022864060878589,
 'P(inf|season) train PR': 0.4441163639127461}